# Oppretting av svarinngangtabell

## OBS! Før kjøring, legg inn eget brukernavn for oppkogling til DYNAREV
## Kjør hele notebooken på en gang! (Velg "Cell" - "Run All")

In [ ]:
import pandas as pd

import sqlite3
from sqlalchemy import Table, create_engine
from sqlalchemy.sql import select
from flask_sqlalchemy import SQLAlchemy

import getpass
import cx_Oracle

from models.models_delt import connect


In [ ]:
import json

with open("config.json") as config: # Laster in valg fra config.json
    config = json.load(config)

# Kobler til dynarev / oracle
#### Skriv inn eget brukernavn

In [ ]:
bruker = getpass.getuser()[:3]
db = cx_Oracle.connect(bruker,getpass.getpass(), "DB1P")

In [ ]:
"""
def connect():
    conn = sqlite3.connect(f"{config['data']['filsti']}/edith.sqlite", timeout=15) #Må endres hvis koblingen skal være opp mot dynarev
    engine = create_engine(f"sqlite:///{config['data']['filsti']}/edith.sqlite")
    #conn = sqlite3.connect("/home/db/fou_sas.sqlite", timeout=15) #Må endres hvis koblingen skal være opp mot dynarev
    #engine = create_engine("sqlite:////home/db/fou_sas.sqlite")
    db = SQLAlchemy()
    return conn, engine, db
"""
conn, engine, db_sqlite = connect()


### Leser og henter inn tabeller fra Dynarev

In [ ]:
data = pd.read_sql(f"SELECT ENHETS_ID, FELT_ID, FELT_VERDI FROM DYNAREV.VW_SKJEMA_DATA WHERE DELREG_NR = {config['perioder']['t']['delreg']}", con=db)
rev = pd.read_sql(f"SELECT ENHETS_ID, REV_KODE, INN_DATO, REV_DATO FROM DYNAREV.VW_SKJEMA_ENHET WHERE DELREG_NR = {config['perioder']['t']['delreg']}", con=db)

In [ ]:
revtabell = data.merge(rev, how='left', on='ENHETS_ID')
revtabell['REV_KODE'] = revtabell['REV_KODE'].fillna('U')
revtabell['FELT_VERDI'] = pd.to_numeric(revtabell['FELT_VERDI'], errors='coerce')
revtabell['FELT_VERDI'] = revtabell['FELT_VERDI'].fillna(0)
#revtabell.head()

### Laster opp oppdatert tabell

In [ ]:
revtabell.to_sql(f'{config["tabeller"]["svarinngang"]}', con = engine, if_exists='replace')

### Laster inn tabell og sjekker


In [ ]:
revtabell = pd.read_sql("SELECT * FROM svarinngang WHERE FELT_ID = 'INTFOU'", con=engine)

In [ ]:
#revtabell